# Amazon SageMaker AI MLOps: from idea to production in six steps

This sequence of six notebooks takes you from developing your ML idea in a simple notebook to a production solution with automated model building and CI/CD deployment pipelines, and model monitoring.

Follow these steps one by one:
**From idea to production in six steps:**
||||
|---|---|---|
|1. |Experiment in a notebook ||
|2. |Scale with SageMaker AI processing jobs and SageMaker SDK ||
|3. |Operationalize with ML pipeline, model registry, and feature store ||
|4. |Add a model building CI/CD pipeline ||
|5. |Add a model deployment pipeline ||
|6. |Add model and data monitoring ||

There are also additional hands-on examples of other SageMaker features and ML topics, like [A/B testing](https://docs.aws.amazon.com/sagemaker/latest/dg/model-validation.html), custom [processing](https://docs.aws.amazon.com/sagemaker/latest/dg/build-your-own-processing-container.html), [training](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html) and [inference](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-main.html) containers, [debugging and profiling](https://docs.aws.amazon.com/sagemaker/latest/dg/train-debugger.html), [security](https://docs.aws.amazon.com/sagemaker/latest/dg/security.html), [multi-model](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-model-endpoints.html) and [multi-container](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-container-endpoints.html) endpoints, and [serial inference pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html). Explore the notebooks in the folder `additional-topics` to test out these features.

To run this notebook and all notebooks in the workshop please use the `Python 3` kernel in JupyterLab

## Star GitHub repository

In [1]:
%%html

<a class="github-button" href="https://github.com/aws-samples/amazon-sagemaker-from-idea-to-production" data-color-scheme="no-preference: light; light: light; dark: dark;" data-icon="octicon-star" data-size="large" data-show-count="true" aria-label="Star Amazon SageMaker secure MLOps on GitHub">Star</a>
<script async defer src="https://buttons.github.io/buttons.js"></script>

### Click this button ^^^ above ^^^

## Setup
Get the latest version of SageMaker Python SDK.

<div class="alert alert-info"> 💡 The workshop and all notebooks were tested on the SageMaker Distribution Image version <strong>3.4.2</strong>. If you encounter any incompatibility issues, you can stop the JupyterLab space and re-start it with a specific version of the SageMaker Distribution Image.
</div>

![](img/jupyterlab-app-image.png)

In [2]:
%pip install --upgrade pip sagemaker boto3

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install --upgrade "mlflow>=2,<3" sagemaker-mlflow

  Using cached mlflow-2.22.2-py3-none-any.whl.metadata (30 kB)
  Using cached sagemaker_mlflow-0.1.1-py3-none-any.whl.metadata (3.6 kB)
  Using cached mlflow_skinny-2.22.2-py3-none-any.whl.metadata (31 kB)
Using cached mlflow-2.22.2-py3-none-any.whl (29.0 MB)
Using cached mlflow_skinny-2.22.2-py3-none-any.whl (6.3 MB)
Using cached sagemaker_mlflow-0.1.1-py3-none-any.whl (15 kB)
  Attempting uninstall: mlflow-skinny
    Found existing installation: mlflow-skinny 2.22.0
    Uninstalling mlflow-skinny-2.22.0:
      Successfully uninstalled mlflow-skinny-2.22.0m 0/3 [mlflow-skinny]
  Attempting uninstall: mlflow━━━━━━━━━━━━━━━━━━ 0/3 [mlflow-skinny]
    Found existing installation: mlflow 2.22.00m 0/3 [mlflow-skinny]
    Uninstalling mlflow-2.22.0:━━━━━━━━━━━━━ 0/3 [mlflow-skinny]
      Successfully uninstalled mlflow-2.22.0 0/3 [mlflow-skinny]
  Attempting uninstall: sagemaker-mlflow━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [mlflow]
    Found existing installation: sagemaker-mlflow 0.1.0━━━━━━━ 1/3 

In [5]:
%pip install -q "sagemaker[local]"

Note: you may need to restart the kernel to use updated packages.


### Import packages

In [6]:
import time
import os
import json
import boto3
import numpy as np  
import pandas as pd 
import sagemaker
from time import gmtime, strftime, sleep
import mlflow

(sagemaker.__version__,boto3.__version__, mlflow.__version__)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


('2.253.1', '1.40.55', '2.22.2')

### Set constants

In [7]:
# Get some variables you need to interact with SageMaker service
boto_session = boto3.Session()
region = boto_session.region_name
bucket_name = sagemaker.Session().default_bucket()
bucket_prefix = "from-idea-to-prod/xgboost"  
sm_session = sagemaker.Session()
sm_client = boto_session.client("sagemaker")
sm_role = sagemaker.get_execution_role()
dataset_file_local_path = "data/bank-additional/bank-additional-full.csv"

initialized = True

print(sm_role)

arn:aws:iam::284038851123:role/service-role/AmazonSageMaker-ExecutionRole-20251007T114513


In [8]:
# Store some variables to keep the value between the notebooks
%store bucket_name
%store bucket_prefix
%store sm_role
%store region
%store initialized
%store dataset_file_local_path

Stored 'bucket_name' (str)
Stored 'bucket_prefix' (str)
Stored 'sm_role' (str)
Stored 'region' (str)
Stored 'initialized' (bool)
Stored 'dataset_file_local_path' (str)


### Get domain id
You need this value `domain_id` in many SageMaker Python SDK and boto3 SageMaker API calls. The notebook metadata file contains `domain_id` value. The following code demonstrates how to access the notebook metadata file and get the `domain_id`.

In [8]:
NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
domain_id = None

if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        metadata = json.loads(f.read())
        domain_id = metadata.get('DomainId')
        space_name = metadata.get('SpaceName')
        print(f"SageMaker domain id: {domain_id}")

if not space_name:
    raise Exception(f"Cannot find the current space name. Make sure you run this notebook in a JupyterLab in the SageMaker Studio")
else:
    print(f"Space name: {space_name}")
    
r = sm_client.describe_space(DomainId=domain_id, SpaceName=space_name)
user_profile_name = r['OwnershipSettings']['OwnerUserProfileName']

assert(user_profile_name)
print(f"User profile: {user_profile_name}")

%store domain_id
%store space_name
%store user_profile_name

SageMaker domain id: d-qregfhzsfc2a
Space name: quickstart-default-4go7kk
User profile: default-20251021T105684
Stored 'domain_id' (str)
Stored 'space_name' (str)
Stored 'user_profile_name' (str)


### Get space details
Use SageMaker API `DescribeSpace` call to get space resource specifications.

In [9]:
r = boto3.client('sagemaker').describe_space(DomainId=domain_id, SpaceName=space_name)
resource_spec = r['SpaceSettings']['JupyterLabAppSettings']['DefaultResourceSpec']
sm_image = resource_spec.get('SageMakerImageArn', 'not defined')
sm_image_version = resource_spec.get('SageMakerImageVersionAlias', 'not defined')
print(f"""
SageMaker image: \033[1m{sm_image}\033[0m
SageMaker image version: \033[1m{sm_image_version}\033[0m
""")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 r = boto3.client('sagemaker').describe_space(DomainId=domain_id, SpaceName=space_name)       │
│ ❱ 2 resource_spec = r['SpaceSettings']['JupyterLabAppSettings']['DefaultResourceSpec']           │
│   3 sm_image = resource_spec.get('SageMakerImageArn', 'not defined')                             │
│   4 sm_image_version = resource_spec.get('SageMakerImageVersionAlias', 'not defined')            │
│   5 print(f"""                                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'JupyterLabAppSettings'

In [10]:
r

{'DomainId': 'd-qregfhzsfc2a',
 'SpaceArn': 'arn:aws:sagemaker:us-east-2:284038851123:space/d-qregfhzsfc2a/quickstart-default-4go7kk',
 'SpaceName': 'quickstart-default-4go7kk',
 'Status': 'InService',
 'LastModifiedTime': datetime.datetime(2025, 10, 21, 17, 0, 50, 770000, tzinfo=tzlocal()),
 'CreationTime': datetime.datetime(2025, 10, 21, 17, 0, 45, 702000, tzinfo=tzlocal()),
 'SpaceSettings': {'AppType': 'JupyterLab',
  'SpaceStorageSettings': {'EbsStorageSettings': {'EbsVolumeSizeInGb': 5}}},
 'OwnershipSettings': {'OwnerUserProfileName': 'default-20251021T105684'},
 'SpaceSharingSettings': {'SharingType': 'Private'},
 'Url': 'https://hriif7dzuohbcbg.studio.us-east-2.sagemaker.aws/jupyterlab/default',
 'ResponseMetadata': {'RequestId': 'cd8b90ff-8ba4-4714-b159-70121db705fa',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'cd8b90ff-8ba4-4714-b159-70121db705fa',
   'strict-transport-security': 'max-age=47304000; includeSubDomains',
   'x-frame-options': 'DENY',
   'con

In [11]:
%store sm_image
%store sm_image_version

UsageError: Unknown variable 'sm_image'


### Connect to MLflow tracking server
If you're running an AWS-led workshop or used the delivered CloudFormation template to provision your workshop environment, an MLflow server must be up and running. If you don't have an MLflow server, follow the [Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow-create-tracking-server.html) or run the following code cell to create a new one.

To create and manage an MLflow tracking server and to work with managed MLflow experiements, you need the following permissions attached to the SageMaker execution role:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker-mlflow:*",
                "sagemaker:CreateMlflowTrackingServer",
                "sagemaker:UpdateMlflowTrackingServer",
                "sagemaker:DeleteMlflowTrackingServer",
                "sagemaker:StartMlflowTrackingServer",
                "sagemaker:StopMlflowTrackingServer",
                "sagemaker:CreatePresignedMlflowTrackingServerUrl"
            ],
            "Resource": "*"
        }
    ]
}
```

Execute the following code to check if you have a running MLflow server.

In [12]:
# Find an active MLflow server in the account
tracking_servers = [s['TrackingServerArn'] for s 
                    in boto3.client("sagemaker").list_mlflow_tracking_servers()['TrackingServerSummaries']
                    if s['IsActive'] == 'Active']

if len(tracking_servers) < 1:
    print("You don't have any active MLflow servers. Trying to find a server in the status 'Creating'...")

    r = boto3.client("sagemaker").list_mlflow_tracking_servers(
        TrackingServerStatus='Creating',
    )['TrackingServerSummaries']

    if len(r) < 1:
        print("You don't have any MLflow server in the status 'Creating'. Run the next code cell to create a new one.")
        mlflow_arn = None
        mlflow_name = None
    else:
        mlflow_arn = r[0]['TrackingServerArn']
        mlflow_name = r[0]['TrackingServerName']
        print(f"You have an MLflow server {mlflow_arn} in the status 'Creating', going to use this one")
else:
    mlflow_arn = tracking_servers[0]
    mlflow_name = tracking_servers[0].split('/')[1]
    print(f"You have {len(tracking_servers)} running MLflow server(s). Get the first server ARN:{mlflow_arn}")

You have 1 running MLflow server(s). Get the first server ARN:arn:aws:sagemaker:us-east-2:284038851123:mlflow-tracking-server/mlflow-d-21qjmpg5nbfo-16-17-36-05


In [11]:
# This code cell creates a new MLflow server
if not mlflow_arn:
    ts = strftime('%d-%H-%M-%S', gmtime())
    mlflow_name = f"mlflow-{domain_id}-{ts}"
    r = boto3.client("sagemaker").create_mlflow_tracking_server(
        TrackingServerName=mlflow_name,
        ArtifactStoreUri=f"s3://{bucket_name}/mlflow/{ts}",
        RoleArn=sm_role,
        AutomaticModelRegistration=True,
    )

    mlflow_arn = r['TrackingServerArn']
    print(f"Server creation request succeded. The server {mlflow_arn} is being created.")

<div style="border: 4px solid coral; text-align: center; margin: auto;">
Creation of an MLflow server can take up to 25 minutes. You don't need to wait - proceed with the flow of the workshop.
</div>

In [13]:
(mlflow_arn, mlflow_name, mlflow.__version__)

('arn:aws:sagemaker:us-east-2:284038851123:mlflow-tracking-server/mlflow-d-21qjmpg5nbfo-16-17-36-05',
 'mlflow-d-21qjmpg5nbfo-16-17-36-05',
 '2.22.2')

In [14]:
%store mlflow_arn
%store mlflow_name

Stored 'mlflow_arn' (str)
Stored 'mlflow_name' (str)


## Install Docker to enable Studio local mode
Amazon SageMaker Studio applications support the use of local mode to create estimators, processors, and pipelines, then deploy them to a local environment. With local mode, you can test machine learning scripts before running them in Amazon SageMaker managed training or hosting environments. Refer to [Local mode support in Amazon SageMaker Studio](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-local.html) to understand which docker operations the Studio currently supports.

To use local mode in Studio applications, you must install Docker into your JupyterLab space. 

### Check if docker access is enabled

In [15]:
# check that docker enabled in the SageMaker domain
docker_settings = sm_client.describe_domain(DomainId=domain_id).get('DomainSettings', {}).get('DockerSettings')
docker_enabled = False

if docker_settings:
    if docker_settings.get('EnableDockerAccess') in ['ENABLED']:
        print(f"The docker access is ENABLED in the domain {domain_id}")
        docker_enabled = True

if not docker_enabled:
    raise Exception(f"You must enable docker access in the domain to use Studio local mode")

The docker access is ENABLED in the domain d-qregfhzsfc2a


<div style="border: 4px solid coral; text-align: center; margin: auto;">
If the previous code cell raised an exeption that the docker access is not enabled, you need to enable the access. See the following instructions how to do it.
</div>

In [16]:
print(f"Domain id: {domain_id}")

Domain id: d-qregfhzsfc2a


### Enable docker access for the SageMaker domain

<div class="alert alert-info">You only need to execute this section if the docker access is not enabled in the domain.
</div>

You need `sagemaker:UpdateDomain` permission in the execution role. You cannot update domain from this notebook because the notebook execution role doesn't have this permission. To update domain settings, you can use one of the following options.

#### Option 1: run `update_domain` in the notebook
If you have the corresponding permissions in the notebook execution role, you can run the following code in a notebook:

```python
import boto3

r = boto3.client('sagemaker').update_domain(
    DomainId=domain_id,
    DomainSettingsForUpdate={
        'DockerSettings': {
            'EnableDockerAccess':'ENABLED',
        }
    }
)
```

#### Option 2: run `aws sagemaker` CLI in the  terminal
Make sure you run `AWS CLI` in the terminal where you have the corresponding permissions `sagemaker:UpdateDomain`. Run the following command:

```
aws sagemaker update-domain --domain-id <DOMAIN-ID> --domain-settings-for-update DockerSettings={EnableDockerAccess='ENABLED'}
```

For example, you can run the command above in the [AWS CloudShell](https://aws.amazon.com/blogs/aws/aws-cloudshell-command-line-access-to-aws-resources/) in your AWS account.

In [17]:
# check the updated settings
sm_client.describe_domain(DomainId=domain_id)['DomainSettings']

{'DockerSettings': {'EnableDockerAccess': 'ENABLED',
  'VpcOnlyTrustedAccounts': []}}

### Install Docker

In [18]:
%%bash

# see https://docs.docker.com/engine/install/ubuntu/#install-using-the-repository
sudo apt-get update
sudo apt-get install -y ca-certificates curl
sudo install -m 0755 -d /etc/apt/keyrings
sudo curl -fsSL https://download.docker.com/linux/ubuntu/gpg -o /etc/apt/keyrings/docker.asc
sudo chmod a+r /etc/apt/keyrings/docker.asc

# Add the repository to Apt sources:
echo \
  "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu \
  $(. /etc/os-release && echo "$VERSION_CODENAME") stable" | \
  sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get update

## Currently only Docker version 20.10.X is supported in Studio: see https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-local.html
# pick the latest patch from:
# apt-cache madison docker-ce | awk '{ print $3 }' | grep -i 20.10
VERSION_STRING=5:20.10.24~3-0~ubuntu-jammy
sudo apt-get install docker-ce-cli=$VERSION_STRING docker-compose-plugin -y

# validate the Docker Client is able to access Docker Server at [unix:///docker/proxy.sock]
docker version

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1288 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6043 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3452 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5803 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3796 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [60.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [69.2 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1594 kB]
Get:13 http://arch

debconf: delaying package configuration, since apt-utils is not installed


Fetched 484 kB in 1s (597 kB/s)
(Reading database ... 14526 files and directories currently installed.)
Preparing to unpack .../curl_7.81.0-1ubuntu1.21_amd64.deb ...
Unpacking curl (7.81.0-1ubuntu1.21) over (7.81.0-1ubuntu1.20) ...
Preparing to unpack .../libcurl4_7.81.0-1ubuntu1.21_amd64.deb ...
Unpacking libcurl4:amd64 (7.81.0-1ubuntu1.21) over (7.81.0-1ubuntu1.20) ...
Setting up libcurl4:amd64 (7.81.0-1ubuntu1.21) ...
Setting up curl (7.81.0-1ubuntu1.21) ...
Processing triggers for libc-bin (2.35-0ubuntu3.10) ...
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:2 https://download.docker.com/linux/ubuntu jammy InRelease [48.5 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Get:5 https://download.docker.com/linux/ubuntu jammy/stable amd64 Packages [69.9 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 118 kB in 1s (192 kB/s)
Reading package lists...
Reading package

debconf: delaying package configuration, since apt-utils is not installed


Fetched 57.5 MB in 1s (81.8 MB/s)
Selecting previously unselected package docker-ce-cli.
(Reading database ... 14526 files and directories currently installed.)
Preparing to unpack .../docker-ce-cli_5%3a20.10.24~3-0~ubuntu-jammy_amd64.deb ...
Unpacking docker-ce-cli (5:20.10.24~3-0~ubuntu-jammy) ...
Selecting previously unselected package docker-compose-plugin.
Preparing to unpack .../docker-compose-plugin_2.40.1-1~ubuntu.22.04~jammy_amd64.deb ...
Unpacking docker-compose-plugin (2.40.1-1~ubuntu.22.04~jammy) ...
Setting up docker-compose-plugin (2.40.1-1~ubuntu.22.04~jammy) ...
Setting up docker-ce-cli (5:20.10.24~3-0~ubuntu-jammy) ...
Client:
 Version:           unknown-version
 API version:       1.44 (downgraded from 1.47)
 Go version:        go1.23.5
 Git commit:        unknown-commit
 Built:             unknown-buildtime
 OS/Arch:           linux/amd64
 Context:           default

Server:
 Engine:
  Version:          25.0.8
  API version:      1.44 (minimum version 1.24)
  Go vers

## Data

This example uses the [direct marketing dataset](https://archive.ics.uci.edu/ml/datasets/bank+marketing) from UCI's ML Repository:
> [Moro et al., 2014] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed.

Download and unzip the dataset:

In [19]:
!wget -P data/ -N https://archive.ics.uci.edu/static/public/222/bank+marketing.zip --no-check-certificate

--2025-10-21 17:21:20--  https://archive.ics.uci.edu/static/public/222/bank+marketing.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘data/bank+marketing.zip’

bank+marketing.zip      [  <=>               ] 999.85K  2.75MB/s    in 0.4s    

Last-modified header missing -- time-stamps turned off.
2025-10-21 17:21:21 (2.75 MB/s) - ‘data/bank+marketing.zip’ saved [1023843]



In [20]:
import zipfile

with zipfile.ZipFile("data/bank+marketing.zip", "r") as z:
    print("Unzipping bank+marketing...")
    z.extractall("data")

with zipfile.ZipFile("data/bank-additional.zip", "r") as z:
    print("Unzipping bank-additional...")
    z.extractall("data")

print("Done")

Unzipping bank+marketing...
Unzipping bank-additional...
Done


### See the data

In [21]:
df_data = pd.read_csv(dataset_file_local_path, sep=";")

pd.set_option("display.max_columns", 500)  # View all of the columns
df_data  # show first 5 and last 5 rows of the dataframe

age          job  marital            education  default housing loan  \
0       56    housemaid  married             basic.4y       no      no   no   
1       57     services  married          high.school  unknown      no   no   
2       37     services  married          high.school       no     yes   no   
3       40       admin.  married             basic.6y       no      no   no   
4       56     services  married          high.school       no      no  yes   
...    ...          ...      ...                  ...      ...     ...  ...   
41183   73      retired  married  professional.course       no     yes   no   
41184   46  blue-collar  married  professional.course       no      no   no   
41185   56      retired  married    university.degree       no     yes   no   
41186   44   technician  married  professional.course       no      no   no   
41187   74      retired  married  professional.course       no     yes   no   

         contact month day_of_week  duration  campaign  pdays  previous  \
0      telephone   may         mon       261         1    999         0   
1      telephone   may         mon       149         1    999         0   
2      telephone   may         mon       226         1    999         0   
3      telephone   may         mon       151         1    999         0   
4      telephone   may         mon       307         1    999         0   
...          ...   ...         ...       ...       ...    ...       ...   
41183   cellular   nov         fri       334         1    999         0   
41184   cellular   nov         fri       383         1    999         0   
41185   cellular   nov         fri       189         2    999         0   
41186   cellular   nov         fri       442         1    999         0   
41187   cellular   nov         fri       239         3    999         1   

          poutcome  emp.var.rate  cons.price.idx  cons.conf.idx  euribor3m  \
0      nonexistent           1.1          93.994          -36.4      4.857   
1      nonexistent           1.1          93.994          -36.4      4.857   
2      nonexistent           1.1          93.994          -36.4      4.857   
3      nonexistent           1.1          93.994          -36.4      4.857   
4      nonexistent           1.1          93.994          -36.4      4.857   
...            ...           ...             ...            ...        ...   
41183  nonexistent          -1.1          94.767          -50.8      1.028   
41184  nonexistent          -1.1          94.767          -50.8      1.028   
41185  nonexistent          -1.1          94.767          -50.8      1.028   
41186  nonexistent          -1.1          94.767          -50.8      1.028   
41187      failure          -1.1          94.767          -50.8      1.028   

       nr.employed    y  
0           5191.0   no  
1           5191.0   no  
2           5191.0   no  
3           5191.0   no  
4           5191.0   no  
...            ...  ...  
41183       4963.6  yes  
41184       4963.6   no  
41185       4963.6   no  
41186       4963.6  yes  
41187       4963.6   no  

[41188 rows x 21 columns]

### Upload data to S3

In [22]:
input_s3_url = sagemaker.Session().upload_data(
    path=dataset_file_local_path,
    bucket=bucket_name,
    key_prefix=f"{bucket_prefix}/input"
)
print(f"Upload the dataset to {input_s3_url}")

%store input_s3_url

Upload the dataset to s3://sagemaker-us-east-2-284038851123/from-idea-to-prod/xgboost/input/bank-additional-full.csv
Stored 'input_s3_url' (str)


## Restart kernel

In [ ]:
# Restart kernel to get the packages
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

## Further workshop flow
You can continue the workshop by going through each of the following notebooks in the direct order, e.g. 1-2-3-4....

If you're interested in a particular topic, you can execute some notebooks standalone, as shown in the following flow diagram:

![](img/workshop-flow.png)

Start with the step 1 [Idea Development](01-idea-development.ipynb) or step 3 [SageMaker Pipelines](03-sagemaker-pipeline.ipynb).

## Additional resources

### Documentation
- [Use Amazon SageMaker Built-in Algorithms](https://docs.aws.amazon.com/sagemaker/latest/dg/algos.html)

### Hands-on examples
- [Get started with Amazon SageMaker](https://aws.amazon.com/sagemaker/getting-started/)


### Workshops
- [Amazon SageMaker 101 Workshop](https://catalog.us-east-1.prod.workshops.aws/workshops/0c6b8a23-b837-4e0f-b2e2-4a3ffd7d645b/en-US)